# 🛠️ Proceso de Transformación de Datos

En este notebook se realiza la **limpieza, normalización y transformación** de los datos extraídos para dejarlos listos para su análisis o carga.

## ✳️ Librerías necesarias


In [ ]:
import pandas as pd
import re
import os

## Inscrito


In [ ]:

df_inscrito = pd.read_csv("/Users/monky02/Desktop/FP/Data/Source/DataOrigin/alr_inscritos.csv")
df_inscrito.head()

,id,nombres,apellidos,dni,pais,ciudad,institucion,profesion,cargo,telefono,...,direccion,correo,skype,grupo_trabajo,det1,det2,nombre_fac,ruc_fac,direccion_fac,telf_fac
0,4,Adriano,Fiorese,74347233949,Brasil,Joinville,Universidade do Estado de Santa Catarina - UDESC,Professor,Professor UniversitÃ¡rio,0055 47 3481 7991,...,"Rua Paulo Malschitzki, 200 Zona Industrial Nor...",adriano.fiorese@udesc.br,af_udesc,NaN,NaN,0,NaN,NaN,NaN,NaN
1,6,Patricia MarÃ­a,HenrÃ­quez Coronel,1757101678,Ecuador,Manta,Universidad Laica Eloy Alfaro de Manabi,PhD en PedagogÃ­a,Profesora Titular Principal,+5930969142352,...,Ciudadela Altos de Manta Beach Torre 6 Dpto 2,patricia.henriquez@uleam.edu.ec,patricia henriquez-coronel,InvestigaciÃ³n en EducaciÃ³n y TecnologÃ­a - I...,NaN,0,Patricia Henriquez C.,1757101678,Manta,0969142352
2,7,Patricia MarÃ­a,HenrÃ­quez Coronel,1757101678,Ecuador,Manta,Universidad Laica Eloy Alfaro de Manabi,PhD en PedagogÃ­a,Profesora Titular Principal,+5930969142352,...,Ciudadela Altos de Manta Beach Torre 6 Dpto 2,patricia.henriquez@uleam.edu.ec,patricia henriquez-coronel,InvestigaciÃ³n en EducaciÃ³n y TecnologÃ­a - I...,NaN,0,Patricia Henriquez C.,1757101678,Manta,0969142352
3,8,CÃ©sar Miguel,Oviedo Salinas,4385470,Paraguay,Villeta,Facultad de Ciencias QuÃ­micas,Ingeniero QuÃ­mico,Docente,021585562,...,Ciudad Universitaria,coviedo@qui.una.py,cesarmiguelo28@hotmail.com,Universidad Nacional de AsunciÃ³n,NaN,0,Facultad de Ciencias QuÃ­micas,80010887-6,Campus Universitario,021585562
4,9,JOSILENE,MOREIRA,759165,Brasil,Joao Pessoa,Universidade Federal da ParaÃ­ba,Analista de Sistemas,Professora,+351926172149,...,"Av. Cabo Branco, 3320",josileneaires@gmail.com,NaN,NaN,NaN,0,JOSILENE AIRES MOREIRA,Projeto Meninas,UFPB - Av. dos Escoteiros s/n - Joao Pessoa - PB,NaN


In [ ]:


archivo_original = '/Users/monky02/Desktop/FP/DataOrigin/alr_inscritos.csv'
carpeta_salida = '/Users/monky02/Desktop/FP/DataLimpiaNormalizada'
nombre_archivo_salida = 'alr_inscritos_limpio.csv'

os.makedirs(carpeta_salida, exist_ok=True)
archivo_salida = os.path.join(carpeta_salida, nombre_archivo_salida)

df = pd.read_csv(archivo_original, encoding='utf-8')

reemplazos = {
    'Ã¡': 'a',
    'Ã©': 'e',
    'Ã­': 'i',
    'Ã³': 'o',
    'Ãº': 'u',
    'Ã±': 'n',
    'Ã': 'A',
    'Ã‰': 'E',
    'Ã': 'I',
    'Ã“': 'O',
    'Ãš': 'U',
    'Ã': 'N',
    'Â': '',
    'N§N£o': 'Não',
    'ã': 'a',
}

def limpiar_texto(texto):
    if pd.isnull(texto) or str(texto).strip().lower() in ['null', '']:
        return 'null'
    texto = str(texto)
    for mal, bien in reemplazos.items():
        texto = texto.replace(mal, bien)
    return texto

def capitalizar(texto):
    if pd.isnull(texto) or str(texto).strip().lower() in ['null', '']:
        return 'null'
    texto = str(texto).strip()
    return texto[0].upper() + texto[1:].lower() if texto else 'null'

def normalizar_telefono(tel):
    if pd.isnull(tel) or str(tel).strip().lower() in ['null', '']:
        return 'null'
    tel_str = str(tel).strip()
    tel_str = re.sub(r'[^\d+]', '', tel_str)
    if '+' in tel_str[1:]:
        tel_str = tel_str.replace('+', '')
    return tel_str if tel_str else 'null'

def normalizar_ruc(valor):
    if pd.isnull(valor) or str(valor).strip().lower() in ['null', '']:
        return 'null'
    val = str(valor).strip()
    val = val.replace('.', '').replace(' ', '')
    return val if val else 'null'

# Limpiar caracteres en todas columnas de texto
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(limpiar_texto)

# Capitalizar texto en todas columnas de texto
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(capitalizar)

# Normalizar columnas específicas
if 'telefono' in df.columns:
    df['telefono'] = df['telefono'].apply(normalizar_telefono)
if 'celular' in df.columns:
    df['celular'] = df['celular'].apply(normalizar_telefono)
if 'ruc_fac' in df.columns:
    df['ruc_fac'] = df['ruc_fac'].apply(normalizar_ruc)

# Validar correos: si no contiene '@', reemplazar por 'null'
if 'correo' in df.columns:
    df['correo'] = df['correo'].apply(lambda x: x if '@' in str(x) else 'null')

# Convertir todas las columnas a string excepto det1 y det2, y reemplazar vacíos por 'null'
columnas_no_tocar = ['det1', 'det2']
for col in df.columns:
    if col not in columnas_no_tocar:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].replace({'': 'null', 'nan': 'null', 'None': 'null'})

df = df.fillna('null')
# Ver resultado limpio
print(df.head())

# Guardar archivo limpio
df.to_csv(archivo_salida, index=False, encoding='utf-8')


print(f"Archivo limpio guardado como: {archivo_salida}")



In [6]:

df_InscritosLimpia = pd.read_csv("/Users/monky02/Desktop/FP/Data/Source/DataLimpiaNormalizada/alr_inscritos_limpio.csv", encoding='utf-8')
print(df_InscritosLimpia.head(),
    
      "\nTipos de datos:\n", df_InscritosLimpia.dtypes)

   id         nombres          apellidos          dni      pais       ciudad  \
0   4         Adriano            Fiorese  74347233949    Brasil    Joinville   
1   6  Patricia maria  Henriquez coronel   1757101678   Ecuador        Manta   
2   7  Patricia maria  Henriquez coronel   1757101678   Ecuador        Manta   
3   8    Cesar miguel     Oviedo salinas      4385470  Paraguay      Villeta   
4   9        Josilene            Moreira       759165    Brasil  Joao pessoa   

                                        institucion             profesion  \
0  Universidade do estado de santa catarina - udesc             Professor   
1           Universidad laica eloy alfaro de manabi      Phd en pedagogia   
2           Universidad laica eloy alfaro de manabi      Phd en pedagogia   
3                     Facultad de ciencias quimicas     Ingeniero quimico   
4                   Universidade federal da paraiba  Analista de sistemas   

                         cargo      telefono  ...  \
0  

## Asignatura

In [ ]:

# Cargar archivo
df_asignatura = pd.read_csv("/Users/monky02/Desktop/FP/DataOrigin/asignatura.csv", encoding='utf-8')

# Diccionario de reemplazo para caracteres mal codificados
reemplazos = {
    'Ã¡': 'a',
    'Ã©': 'e',
    'Ã­': 'i',
    'Ã³': 'o',
    'Ãº': 'u',
    'Ã±': 'n',
    'Ã': 'A',
    'Ã‰': 'E',
    'Ã': 'I',
    'Ã“': 'O',
    'Ãš': 'U',
    'Ã': 'N',
    'Â': '',
    'N§N£o': 'Não',
    'ã': 'a',
}

def limpiar_texto(texto):
    if pd.isnull(texto) or str(texto).strip().lower() in ['null', '']:
        return 'null'
    texto = str(texto)
    for mal, bien in reemplazos.items():
        texto = texto.replace(mal, bien)
    return texto.strip()

def capitalizar(texto):
    if texto == 'null':
        return texto
    return texto[0].upper() + texto[1:].lower()

# Aplicar limpieza en todas las columnas tipo texto
for col in df_asignatura.select_dtypes(include='object').columns:
    df_asignatura[col] = df_asignatura[col].apply(limpiar_texto)
    df_asignatura[col] = df_asignatura[col].apply(capitalizar)

# Convertir vacíos y NaN restantes a 'null'
df_asignatura = df_asignatura.fillna('null')
df_asignatura = df_asignatura.replace({'': 'null', 'nan': 'null', 'None': 'null'})

# Ver resultado limpio
print(df_asignatura.head())

# Guardar
df_asignatura.to_csv("/Users/monky02/Desktop/FP/DataLimpiaNormalizada/asignatura_limpia.csv", index=False, encoding='utf-8')
print(" Archivo limpio guardado como asignatura_limpia.csv")


In [10]:
df_AsignaturaLimpia = pd.read_csv("/Users/monky02/Desktop/FP/Data/Source/DataLimpiaNormalizada/asignatura_limpia.csv", encoding='utf-8')
print(df_AsignaturaLimpia.head(),
    
      "\nTipos de datos:\n", df_AsignaturaLimpia.dtypes)



   idAsignatura     codigo                              nombre
0             5  Utpldg001                         Prototipado
1             6  Utplrl001                      Ingenieria web
2             7  Utplrc001  Procesos de ingenieria de software
3             8  Utplfc001              Computacion y sociedad
4             9  Utpldi001                Estructuras de datos 
Tipos de datos:
 idAsignatura     int64
codigo          object
nombre          object
dtype: object


## Docente

In [ ]:


# Cargar archivo
df_docente = pd.read_csv("/Users/monky02/Desktop/FP/DataOrigin/docente.csv", encoding='utf-8')

# Reemplazos comunes de errores de codificación
reemplazos = {
    'Ã¡': 'a', 'Ã©': 'e', 'Ã­': 'i', 'Ã³': 'o', 'Ãº': 'u', 'Ã±': 'n',
    'Ã': 'A', 'Ã‰': 'E', 'Ã': 'I', 'Ã“': 'O', 'Ãš': 'U',
    'Ã': 'N', 'Â': '', 'ã': 'a'
}

def limpiar_texto(texto):
    if pd.isnull(texto) or str(texto).strip().lower() in ['null', '', 'nan']:
        return 'null'
    texto = str(texto)
    for mal, bien in reemplazos.items():
        texto = texto.replace(mal, bien)
    return texto.strip()

def capitalizar(texto):
    if texto == 'null':
        return texto
    texto = texto.strip()
    return texto[0].upper() + texto[1:].lower() if texto else 'null'

def validar_correo(correo):
    if pd.isnull(correo) or '@' not in str(correo):
        return 'null'
    return str(correo).strip()

def limpiar_identificacion(ident):
    if pd.isnull(ident):
        return 'null'
    ident_str = re.sub(r'\D', '', str(ident))  # Solo dígitos
    return ident_str if ident_str else 'null'

# Aplicar limpieza
columnas_texto = df_docente.select_dtypes(include='object').columns
for col in columnas_texto:
    df_docente[col] = df_docente[col].apply(limpiar_texto)

# Capitalizar nombres (si aplica)
if 'nombres' in df_docente.columns:
    df_docente['nombres'] = df_docente['nombres'].apply(capitalizar)

# Validar correos
if 'correo' in df_docente.columns:
    df_docente['correo'] = df_docente['correo'].apply(validar_correo)

# Limpiar identificaciones
if 'identificacion' in df_docente.columns:
    df_docente['identificacion'] = df_docente['identificacion'].apply(limpiar_identificacion)

# Convertir vacíos y NaN a 'null'
df_docente = df_docente.fillna('null')
df_docente = df_docente.replace({'': 'null', 'None': 'null', 'nan': 'null'})

# Guardar archivo limpio
df_docente.to_csv("/Users/monky02/Desktop/FP/DataLimpiaNormalizada/docente_limpio.csv", index=False, encoding='utf-8')
print(" Archivo limpio guardado como docente_limpio.csv")


In [9]:

df_DocenteLimpio = pd.read_csv("/Users/monky02/Desktop/FP/Data/Source/DataLimpiaNormalizada/docente_limpio.csv", encoding='utf-8')
print(df_DocenteLimpio.head(),
    
      "\nTipos de datos:\n", df_DocenteLimpio.dtypes)


   idDocente                                         nombres  identificacion  \
0          7  Ramiro leonardo  ramirez coronel administrador      1104076797   
1          9                                   Daniel guaman         1111111   
2         10                                    Roddy correa        11111111   
3         11                                 Fanny  cevallos          111111   
4         13                                    Daniel irene             111   

                   correo  rol    username                          password  
0   rlramirez@utpl.edu.ec  1.0   rlramirez  6355200384765677651e075debb01ae0  
1    daguaman@utpl.edu.ec  2.0    daguaman  75275e769db3b685af9bd9359be4ac9b  
2   racorrea2@utpl.edu.ec  2.0   racorrea2  8dc4253bede219486fb1741d1f41d214  
3  fbcevallos@utpl.edu.ec  2.0  fbcevallos  a2371c2d05b8dd55701f2472dd6cacaf  
4       pdirene@gmail.com  2.0     pdirene  095382c6221dd36799da3f7a55877bb2   
Tipos de datos:
 idDocente           int64
n

## Docente Asignatura

In [ ]:

# Cargar archivo
df_docente_asignatura = pd.read_csv("/Users/monky02/Desktop/FP/DataOrigin/docenteasignatura.csv", encoding='utf-8')

# Asegurar que las columnas sean numéricas o 'null'
for col in ['id', 'idDocente', 'idAsignatura']:
    df_docente_asignatura[col] = pd.to_numeric(df_docente_asignatura[col], errors='coerce')  # convierte errores a NaN

# Reemplazar NaN por 'null' (si necesitas texto)
df_docente_asignatura = df_docente_asignatura.fillna('null')


# Guardar archivo limpio
df_docente_asignatura.to_csv("/Users/monky02/Desktop/FP/DataLimpiaNormalizada/docenteasignatura_limpio.csv", index=False, encoding='utf-8')
print(" Archivo limpio guardado como docente_asignatura_limpio.csv")


In [12]:

df_Docente_AsignaturaLimpio = pd.read_csv("/Users/monky02/Desktop/FP/Data/Source/DataLimpiaNormalizada/docente_limpio.csv", encoding='utf-8')
print(df_Docente_AsignaturaLimpio.head(),
    
      "\nTipos de datos:\n", df_Docente_AsignaturaLimpio.dtypes)


   idDocente                                         nombres  identificacion  \
0          7  Ramiro leonardo  ramirez coronel administrador      1104076797   
1          9                                   Daniel guaman         1111111   
2         10                                    Roddy correa        11111111   
3         11                                 Fanny  cevallos          111111   
4         13                                    Daniel irene             111   

                   correo  rol    username                          password  
0   rlramirez@utpl.edu.ec  1.0   rlramirez  6355200384765677651e075debb01ae0  
1    daguaman@utpl.edu.ec  2.0    daguaman  75275e769db3b685af9bd9359be4ac9b  
2   racorrea2@utpl.edu.ec  2.0   racorrea2  8dc4253bede219486fb1741d1f41d214  
3  fbcevallos@utpl.edu.ec  2.0  fbcevallos  a2371c2d05b8dd55701f2472dd6cacaf  
4       pdirene@gmail.com  2.0     pdirene  095382c6221dd36799da3f7a55877bb2   
Tipos de datos:
 idDocente           int64
n

## Result Class

In [ ]:


# Cargar archivo
df_dim = pd.read_csv("/Users/monky02/Desktop/FP/DataOrigin/result_class.csv", encoding='utf-8')

# Reemplazos de caracteres
reemplazos = {
    'Ã¡': 'a', 'Ã©': 'e', 'Ã­': 'i', 'Ã³': 'o', 'Ãº': 'u', 'Ã±': 'n',
    'Ã': 'A', 'Ã‰': 'E', 'Ã': 'I', 'Ã“': 'O', 'Ãš': 'U',
    'Ã': 'N', 'Â': '', 'ã': 'a',
}

def limpiar_texto(texto):
    if pd.isnull(texto) or str(texto).strip().lower() in ['null', '', 'nan']:
        return 'null'
    texto = str(texto)
    for mal, bien in reemplazos.items():
        texto = texto.replace(mal, bien)
    return texto.strip()

# Limpiar columnas tipo texto
for col in df_dim.select_dtypes(include='object').columns:
    df_dim[col] = df_dim[col].apply(limpiar_texto)

# Rellenar NaN en columnas numéricas con 0 y convertir a int
columnas_numericas = df_dim.select_dtypes(include=['float64', 'int64']).columns
for col in columnas_numericas:
    df_dim[col] = df_dim[col].fillna(0).astype(int)

# Rellenar cualquier otro vacío por 'null'
df_dim = df_dim.fillna('null')
df_dim = df_dim.replace({'': 'null', 'None': 'null', 'nan': 'null'})

# Guardar archivo limpio
salida = "/Users/monky02/Desktop/FP/DataLimpiaNormalizada/dimensiones_limpio.csv"
os.makedirs(os.path.dirname(salida), exist_ok=True)
df_dim.to_csv(salida, index=False, encoding='utf-8')

print("✅ Archivo limpio guardado como dimensiones_limpio.csv")


In [14]:
df_resullt_classLimpio = pd.read_csv("/Users/monky02/Desktop/FP/Data/Source/DataLimpiaNormalizada/dimensiones_limpio.csv", encoding='utf-8')
print(df_resullt_classLimpio.head(),
    
      "\nTipos de datos:\n", df_resullt_classLimpio.dtypes)

   id  random                fecha                               UIDE  \
0  31    7530  2020-04-14 01:20:28                      Comentarios..   
1  32    6541  2020-04-14 11:07:07                      Comentarios..   
2  33    7530  2020-04-14 11:13:34                      Comentarios..   
3  34    8873  2020-04-14 20:39:10                      Comentarios..   
4  35    8873  2020-04-14 20:39:21  La clase muy buena y entretenida.   

   val_class  senti1  senti2  senti3  senti4  senti5  senti6  senti7  senti8  \
0          1       1       1       1       1       1       1       1       1   
1          4       0       1       0       0       0       0       1       1   
2          0       0       0       1       1       1       0       0       0   
3          1       0       0       1       0       0       0       0       0   
4          1       0       1       0       0       0       0       0       0   

   senti9  senti10              det  
0       0        0  181.113.149.199  
1   

Una vez con la limpieza de los datos y su extraccion procederemos a un analissis ralgunasappido sobre algunas tablas luego estas seran modificadas en una tabla genera;

## Inscritos

In [ ]:


# Suponiendo que ya tienes cargado df_InscritosLimpia

# 1. ¿Cuántos inscritos hay?
total_inscritos = df_InscritosLimpia.shape[0]
print(f"✅ Total de inscritos: {total_inscritos}")

# 2. ¿De qué país son los que más se inscribieron? (Top 5)
top_paises = df_InscritosLimpia['pais'].value_counts().head(5)
print("\n🌍 Top 5 países con más inscritos:")
print(top_paises)

# 3. ¿Cuántos países distintos hay?
total_paises = df_InscritosLimpia['pais'].nunique()
print(f"\n🗺️ Total de países diferentes: {total_paises}")

# 4. ¿Cuál es la institución que más se repite?
institucion_mas_repetida = df_InscritosLimpia['institucion'].mode()[0]
institucion_count = df_InscritosLimpia['institucion'].value_counts()[institucion_mas_repetida]
print(f"\n🏛️ Institución más frecuente: {institucion_mas_repetida} ({institucion_count} veces)")

# 5. ¿Cuántas profesiones distintas hay?
total_profesiones = df_InscritosLimpia['profesion'].nunique()
print(f"\n🧑‍🎓 Total de profesiones diferentes: {total_profesiones}")

# 6. ¿Cuál es la profesión que más se repite?
profesion_mas_repetida = df_InscritosLimpia['profesion'].mode()[0]
profesion_count = df_InscritosLimpia['profesion'].value_counts()[profesion_mas_repetida]
print(f"\n💼 Profesión más común: {profesion_mas_repetida} ({profesion_count} veces)")




✅ Total de inscritos: 736

🌍 Top 5 países con más inscritos:
pais
Brasil        212
Ecuador       105
Costa rica    102
Paraguay       93
Peru           56
Name: count, dtype: int64

🗺️ Total de países diferentes: 27

🏛️ Institución más frecuente: Universidad tecnica particular de loja (50 veces)

🧑‍🎓 Total de profesiones diferentes: 117

💼 Profesión más común: Estudiante (64 veces)


## Asignatura

In [16]:
# 1. Total de asignaturas
print("Total de asignaturas:", df_AsignaturaLimpia.shape[0])

# 2. Asignaturas con nombres únicos
print("Asignaturas únicas:", df_AsignaturaLimpia['nombre'].nunique())

# 3. Cantidad de asignaturas con nombres duplicados
print("Cantidad de asignaturas con nombre duplicado:", df_AsignaturaLimpia.duplicated('nombre').sum())

# 4. ¿Cuál es la asignatura que más se repite (por nombre)?
asignatura_mas_repetida = df_AsignaturaLimpia['nombre'].mode()[0]
conteo_mas_repetida = df_AsignaturaLimpia['nombre'].value_counts()[asignatura_mas_repetida]
print(f"\n📚 Asignatura más repetida: {asignatura_mas_repetida} ({conteo_mas_repetida} veces)")




Total de asignaturas: 390
Asignaturas únicas: 375
Cantidad de asignaturas con nombre duplicado: 15

📚 Asignatura más repetida: Practicum 2 oct21-feb22 (3 veces)


## Docente

In [17]:
# 1. Total de docentes
print("Total de docentes:", df_DocenteLimpio.shape[0])

# 2. Docentes por rol
print("Docentes por rol:")
print(df_DocenteLimpio['rol'].value_counts())


Total de docentes: 232
Docentes por rol:
rol
2.0    230
1.0      1
Name: count, dtype: int64


## Result Class

In [18]:

# Suponiendo que tu DataFrame se llama df_result
# Cambia a df_class si es otro nombre

# 1. ¿Cuántos valores hay en la columna random?
total_random = df_resullt_classLimpio['random'].shape[0]
print(f"✅ Total de registros en 'random': {total_random}")

# 2. ¿Cuántos valores únicos hay en random?
unicos_random = df_resullt_classLimpio['random'].nunique()
print(f"🔹 Valores únicos en 'random': {unicos_random}")

# 3. ¿Cuántos valores repetidos hay en random?
repetidos_random = total_random - unicos_random
print(f"🔁 Valores repetidos en 'random': {repetidos_random}")

# 4. ¿Cuántos valores únicos hay en la columna det?
unicos_det = df_resullt_classLimpio['det'].nunique()
print(f"\n🔹 Valores únicos en 'det': {unicos_det}")

# 5. ¿Cuántos valores repetidos hay en det?
total_det = df_resullt_classLimpio['det'].shape[0]
repetidos_det = total_det - unicos_det
print(f"🔁 Valores repetidos en 'det': {repetidos_det}")


✅ Total de registros en 'random': 13696
🔹 Valores únicos en 'random': 909
🔁 Valores repetidos en 'random': 12787

🔹 Valores únicos en 'det': 5249
🔁 Valores repetidos en 'det': 8447
